In [5]:
!pip install ipynb

In [17]:
import torch
import re
import pandas as pd
import pickle
# from ipynb.fs.full.preprocess import prepros, process
# from preprocess import prepros, process
# from SentimentLSTM import SentimentLSTM

In [18]:
# full_train = pd.read_csv('./input/full_train.csv')


In [19]:
def preprocess(text):
  #bỏ tag html và emoji
  text = re.sub('<[^>]*>', '', text)
  text = deEmojify(text)

  #thay chữ cái viết hoa thành viết thường
  text = text.lower()

  #xóa dấu ngắt câu, xóa link và các chữ có chứa chữ số
  clean_text = []
  punc_list = r'.,;:?!\|/&@`~()-_@#$%^*\'\"'
  for w in (text.split()):
    if "http" in w:
      continue
    clean_text.append(w)
  text = ' '.join(clean_text)
  for punc in punc_list:
    text = text.replace(punc, ' ')

  #xóa bỏ các chữ cái lặp liên tiếp nhau (đỉnhhhhhhhhhh, vipppppppppppppppp)
  length = len(text)
  char = 0
  while char <length-1:
    if text[char] == text[char+1]:
      text = text[:char]+text[char+1:]
      #print(text)
      length-=1
      continue
    char+=1  
  numbers = ["không", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
  #chuyển đổi các từ tiếng anh và viết tắt thông dụng sang tiếng Việt chuẩn:
  text_split = text.split()
  for i, w in enumerate(text_split):
    if w in englishwords.index:
      text_split[i] = str(englishwords.loc[w, "Vietnamese"])
    if w in shortform.index:
      text_split[i] = str(shortform.loc[w, "Long"])
    if w.isdigit():
      text_split[i] = ' '.join([numbers[int(c)] for c in w]) 
  text = ' '.join(text_split)

  #loại bỏ tất cả các kí tự đặc biệt còn lại
  digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ0123456789 '
  text = ''.join([i for i in text if i in digits_and_characters])
  return text
#split all sentences in corpus
def splitCorpus(corpus):
  t = [sentence.split() for sentence in corpus]
  return t
#join all splited sentences to a big text document
def joinAllSplit(tokenized_sentences):
  sentences = [' '.join(sentence) for sentence in tokenized_sentences]
  return ' '.join(sentences)

#below function get performe preprocessing and remove unknown words
def prepros(sentences):
  new_sentences = [preprocess(sentence) for sentence in sentences]
  splitted_sentences = splitCorpus(new_sentences)
  new = []
  for sentence in bigram[splitted_sentences]:
    new_sentence = ' '.join([word for word in sentence if word in word2idx.keys()])
    new.append(new_sentence)
  return new

#convert words to numbers
def sentenceToInt(sentences):
  #print(sentences)
  int_sentences = []
  for sentence in sentences:
    int_sentence = [word2idx[word] for word in sentence.split()]   
    int_sentences.append(int_sentence)
  return int_sentences

#pad int_sentences to the feature_leng
def padFeature(sentences, feature_leng = 50):
  smatrix = np.zeros((len(sentences), feature_leng))
  for sen_index, sentence in enumerate(sentences):
    padding = max(0, feature_leng - len(sentence))
    for word_index in range(feature_leng):
      if word_index < padding:
        smatrix[sen_index, word_index] = 0
      else:
        smatrix[sen_index, word_index] = sentence[word_index-padding]
  return smatrix

def process(sentences, feature_leng = 50):
  int_sentences = sentenceToInt(sentences)
  feature_matrix = padFeature(int_sentences, feature_leng = 50)
  return feature_matrix


In [21]:
#word2idx
word2idx = pickle.load(open("saves/word2idx.pickle", "rb"))

{'dặt': 1, 'rua': 2, 'cha_nil': 3, 'lucy': 4, 'then_chốt': 5, 'nóng': 6, 'trân': 7, 'mỏm_núi': 8, 'nhi_diệc': 9, 'kính_cẩn': 10, 'quát_to': 11, 'youtube': 12, 'thắt_lưng': 13, 'hiềm': 14, 'đen_láy': 15, 'nhuốm': 16, 'viện': 17, 'vatican': 18, 'biện_pháp': 19, 'chạm_trán': 20, 'vùng_vằng': 21, 'ống': 22, 'chập_chờn': 23, 'dới': 24, 'tôlômiet': 25, 'cường': 26, 'vẫn': 27, 'bối_rối': 28, 'ủy_thác': 29, 'cõi': 30, 'kịch': 31, 'moreau': 32, 'trấn': 33, 'dang_dở': 34, 'phố_wal': 35, 'nhem': 36, 'hành': 37, 'trà_trộn': 38, 'robert_langdon': 39, 'cân_nhắc': 40, 'khoang': 41, 'mini': 42, 'trực_thăng': 43, 'lem': 44, 'nhìu': 45, 'vêt': 46, 'định': 47, 'loạn': 48, 'đó': 49, 'chẹt': 50, 'vòng': 51, 'than_vãn': 52, 'diện_mạo': 53, 'trần_trụi': 54, 'tỵ': 55, 'bỡ_ngỡ': 56, 'lạ_lẫm': 57, 'hàn': 58, 'bc': 59, 'chìa': 60, 'đứng': 61, 'rít': 62, 'điếu_thuốc': 63, 'tich': 64, 'giôngđơret': 65, 'muc': 66, 'suồng': 67, 'tot': 68, 'môngpácnát': 69, 'cải_trang': 70, 'ừ_nhỉ': 71, 'chuộng': 72, 'cư': 73, 'cẩm_n